In [ ]:
import os
import cv2
import mediapipe as mp
from concurrent.futures import ThreadPoolExecutor, as_completed

# Folder paths for WSL
train_folder = "/mnt/d/ASL/Dataset/videos/OLD_train"
test_folder = "/mnt/d/ASL/Dataset/videos/OLD_test"
val_folder = "/mnt/d/ASL/Dataset/videos/OLD_val"

# Output folders for WSL
output_train_folder = "/mnt/d/ASL/Dataset/videos/PIPE_train"
output_test_folder = "/mnt/d/ASL/Dataset/videos/PIPE_test"
output_val_folder = "/mnt/d/ASL/Dataset/videos/PIPE_val"

# Ensure output folders exist
for folder in [output_train_folder, output_test_folder, output_val_folder]:
    os.makedirs(folder, exist_ok=True)

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set up video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with MediaPipe (no custom timestamp)
        results = hands.process(frame_rgb)

        # Draw hand landmarks on the frame
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Write processed frame to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Processed video saved at {output_path}")

def process_folder_parallel(input_folder, output_folder):
    # List of video files to process
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        # Dictionary to keep track of futures and their respective video file
        future_to_video = {
            executor.submit(process_video, os.path.join(input_folder, video), os.path.join(output_folder, video)): video
            for video in video_files
        }

        # Track progress of each video processing
        for future in as_completed(future_to_video):
            video_name = future_to_video[future]
            try:
                future.result()
                print(f"Completed processing of {video_name}")
            except Exception as exc:
                print(f"Error processing {video_name}: {exc}")

# Process each folder in parallel
process_folder_parallel(train_folder, output_train_folder)
process_folder_parallel(test_folder, output_test_folder)
process_folder_parallel(val_folder, output_val_folder)

# Clean up MediaPipe resources
hands.close()


2024-11-13 18:58:47.401347: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 18:58:49.347601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731542352.459132   10752 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731542352.483112   10752 inference_feedback_manager.cc:114] Feedback manager requires a model with a sing

Error processing airplane.mp4: Graph has errors: 
; Packet timestamp mismatch on a calculator receiving from stream "image". Current minimum expected timestamp is 133333 but received 133332. Are you using a custom InputStreamHandler? Note that some InputStreamHandlers allow timestamps that are not strictly monotonically increasing. See for example the ImmediateInputStreamHandler class comment.
; Packet timestamp mismatch on a calculator receiving from stream "image". Current minimum expected timestamp is 133333 but received 133332. Are you using a custom InputStreamHandler? Note that some InputStreamHandlers allow timestamps that are not strictly monotonically increasing. See for example the ImmediateInputStreamHandler class comment.
; Packet timestamp mismatch on a calculator receiving from stream "image". Current minimum expected timestamp is 133333 but received 133332. Are you using a custom InputStreamHandler? Note that some InputStreamHandlers allow timestamps that are not strictl